In [24]:
import pandas as pd
import string
import pymorphy2
from nltk.collocations import BigramAssocMeasures, TrigramAssocMeasures
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
import math
from lxml import etree
from nltk.corpus import stopwords

In [18]:
def parse_xml(filename):
    with open(filename, encoding='utf-8') as f:
        xml = f.read()

    dict = {}
    text = []
    category = []
    sentiment = []
    term = []

    root = etree.fromstring(xml)
    for child in root:
        for aspect in child[3]:
            if aspect.attrib['type'] == 'implicit':
                text.append(child[2].text)
                category.append(aspect.attrib['category'])
                sentiment.append(aspect.attrib['sentiment'])
                term.append(aspect.attrib['term'])

    dict['text'] = text
    dict['category'] = category
    dict['sentiment'] = sentiment
    dict['term'] = term

    # df = pd.DataFrame(dict)
    # print (df)
    return set(text)


In [8]:
text_train = parse_xml('SentiRuEval_rest_markup_train.xml')
text_test = parse_xml('SentiRuEval_rest_markup_test.xml')

all_text = ''
for text in text_train:
    all_text += text + ' '

for text in text_test:
    all_text += text + ' ' 
    
all_text

'Добрый день! Были вчера с друзьями в этом кафе! Очень понравилось! Огромное спасибо поварам, очень вкусные блюда!!!!! Заказывали бивштексы, нет слов как вкусно))) Готовят очень быстро и главное качественно, сразу чувствуется, что там работают мастера своего дела! Спасибо огромное за хорошее обслуживание, приятную атмосферу и гостеприимных официантов! Надеемся еще не раз посетить это замечательное, уютное место с великолепной кухней и доброжелательным персоналом. Я была здесь с подругой пока один раз. Пришли вечером, чтобы послушать живую музыку, отдохнуть и пообщаться. Заказали столик заранее. Остались всем довольны. На первом этаже кафешка, а на втором этаже 3 зала. На счет еды не могу сказать точно…За один раз оценить всё невозможно, но то, что заказали, соответствовало своим вкусовым качествам и моему вкусу(заказали роллы Филадельфия, греческий салат и пиво). Меню разнообразное, цены адекватные. Понравился интерьер. Столики есть на любой вкус и с диванами и у окна и напротив сцены.

In [29]:
stop = set(stopwords.words('russian'))
tmp = word_tokenize(all_text)
tokens = []
for i in tmp:
    if i.isalnum() and i not in stop and not i.isdigit():
        tokens.append(i.lower())
tokens

['добрый',
 'день',
 'были',
 'вчера',
 'друзьями',
 'кафе',
 'очень',
 'понравилось',
 'огромное',
 'спасибо',
 'поварам',
 'очень',
 'вкусные',
 'блюда',
 'заказывали',
 'бивштексы',
 'слов',
 'вкусно',
 'готовят',
 'очень',
 'быстро',
 'главное',
 'качественно',
 'сразу',
 'чувствуется',
 'работают',
 'мастера',
 'своего',
 'дела',
 'спасибо',
 'огромное',
 'хорошее',
 'обслуживание',
 'приятную',
 'атмосферу',
 'гостеприимных',
 'официантов',
 'надеемся',
 'посетить',
 'это',
 'замечательное',
 'уютное',
 'место',
 'великолепной',
 'кухней',
 'доброжелательным',
 'персоналом',
 'я',
 'подругой',
 'пока',
 'пришли',
 'вечером',
 'послушать',
 'живую',
 'музыку',
 'отдохнуть',
 'пообщаться',
 'заказали',
 'столик',
 'заранее',
 'остались',
 'всем',
 'довольны',
 'на',
 'первом',
 'этаже',
 'кафешка',
 'втором',
 'этаже',
 'зала',
 'на',
 'счет',
 'еды',
 'могу',
 'сказать',
 'оценить',
 'всё',
 'невозможно',
 'заказали',
 'соответствовало',
 'своим',
 'вкусовым',
 'качествам',
 'моем

In [45]:
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
finder.apply_freq_filter(10)
finder.nbest(bigram_measures.pmi, 30) 

[('всяких', 'похвал'),
 ('молодым', 'человеком'),
 ('выше', 'всяких'),
 ('первом', 'этаже'),
 ('второй', 'этаж'),
 ('что', 'касается'),
 ('высшем', 'уровне'),
 ('живая', 'музыка'),
 ('молодой', 'человек'),
 ('самое', 'главное'),
 ('приятно', 'удивлены'),
 ('обязательно', 'придем'),
 ('данное', 'заведение'),
 ('день', 'рождения'),
 ('могу', 'сказать'),
 ('добрый', 'день'),
 ('отдельное', 'спасибо'),
 ('хочу', 'отметить'),
 ('большой', 'выбор'),
 ('вкусная', 'еда'),
 ('огромное', 'спасибо'),
 ('долго', 'ждать'),
 ('в', 'общем'),
 ('хочу', 'сказать'),
 ('порции', 'большие'),
 ('в', 'целом'),
 ('уютное', 'место'),
 ('еда', 'вкусная'),
 ('остались', 'довольны'),
 ('большие', 'порции')]

In [46]:
finder.nbest(bigram_measures.chi_sq, 30)

[('всяких', 'похвал'),
 ('молодым', 'человеком'),
 ('что', 'касается'),
 ('выше', 'всяких'),
 ('первом', 'этаже'),
 ('второй', 'этаж'),
 ('день', 'рождения'),
 ('молодой', 'человек'),
 ('живая', 'музыка'),
 ('могу', 'сказать'),
 ('высшем', 'уровне'),
 ('в', 'общем'),
 ('отдельное', 'спасибо'),
 ('приятно', 'удивлены'),
 ('данное', 'заведение'),
 ('обязательно', 'придем'),
 ('самое', 'главное'),
 ('добрый', 'день'),
 ('в', 'целом'),
 ('огромное', 'спасибо'),
 ('большой', 'выбор'),
 ('вкусная', 'еда'),
 ('хочу', 'отметить'),
 ('долго', 'ждать'),
 ('остались', 'довольны'),
 ('хочу', 'сказать'),
 ('еда', 'вкусная'),
 ('порции', 'большие'),
 ('большое', 'спасибо'),
 ('уютное', 'место')]

In [47]:
finder.nbest(bigram_measures.likelihood_ratio, 30)

[('в', 'общем'),
 ('день', 'рождения'),
 ('могу', 'сказать'),
 ('молодой', 'человек'),
 ('в', 'целом'),
 ('отдельное', 'спасибо'),
 ('что', 'касается'),
 ('живая', 'музыка'),
 ('всяких', 'похвал'),
 ('молодым', 'человеком'),
 ('очень', 'довольны'),
 ('огромное', 'спасибо'),
 ('очень', 'вкусно'),
 ('очень', 'понравилось'),
 ('добрый', 'день'),
 ('данное', 'заведение'),
 ('выше', 'всяких'),
 ('остались', 'очень'),
 ('большой', 'выбор'),
 ('второй', 'этаж'),
 ('первом', 'этаже'),
 ('приятно', 'удивлены'),
 ('вкусная', 'еда'),
 ('высшем', 'уровне'),
 ('обязательно', 'придем'),
 ('остались', 'довольны'),
 ('это', 'место'),
 ('долго', 'ждать'),
 ('очень', 'понравился'),
 ('самое', 'главное')]

In [52]:
trigram_measures = TrigramAssocMeasures()
finder1 = TrigramCollocationFinder.from_words(tokens)
finder1.apply_freq_filter(5)
finder1 = TrigramCollocationFinder.from_words(tokens, window_size=3)
finder1.nbest(trigram_measures.pmi, 30)

[('le', 'glamure', 'хрустальный'),
 ('mon', 'petit', 'cafe'),
 ('sokos', 'hotel', 'vasilievsky'),
 ('антонина', 'разбирается', 'предсталенной'),
 ('ассорти', 'капусту', 'тушенную'),
 ('бадене', 'живём', 'озерках'),
 ('базар', 'перед', 'уходом'),
 ('беду', 'решаю', 'отвести'),
 ('бекон', 'хрустящем', 'тосте'),
 ('беседки', 'веселые', 'фигурки'),
 ('бесплатной', 'воды', 'бокалам'),
 ('бешеную', 'популярность', 'владельцы'),
 ('быстрого', 'ненавязчивого', 'доброжелательного'),
 ('вернее', 'месторасполажения', 'приличных'),
 ('вес', 'картинках', 'речи'),
 ('веселую', 'программу', 'аниматоры'),
 ('видны', 'китайские', 'аттрибуты'),
 ('включала', 'грецкие', 'орехи'),
 ('воздушные', 'шары', 'требуются'),
 ('возникла', 'картина', 'воспоминаний'),
 ('восточному', 'инжирным', 'вареньем'),
 ('встречаются', 'зазвездившиеся', 'админы'),
 ('вторую', 'смену', 'обидел'),
 ('выполнения', 'церемонии', 'чаяналивания'),
 ('выступали', 'egeiro', 'project'),
 ('высшее', 'наслаждение', 'пробуйте'),
 ('выходи

In [50]:
finder1.nbest(trigram_measures.chi_sq, 30)

[('le', 'glamure', 'хрустальный'),
 ('mon', 'petit', 'cafe'),
 ('sokos', 'hotel', 'vasilievsky'),
 ('антонина', 'разбирается', 'предсталенной'),
 ('ассорти', 'капусту', 'тушенную'),
 ('бадене', 'живём', 'озерках'),
 ('базар', 'перед', 'уходом'),
 ('беду', 'решаю', 'отвести'),
 ('бекон', 'хрустящем', 'тосте'),
 ('беседки', 'веселые', 'фигурки'),
 ('бесплатной', 'воды', 'бокалам'),
 ('бешеную', 'популярность', 'владельцы'),
 ('быстрого', 'ненавязчивого', 'доброжелательного'),
 ('вернее', 'месторасполажения', 'приличных'),
 ('вес', 'картинках', 'речи'),
 ('веселую', 'программу', 'аниматоры'),
 ('видны', 'китайские', 'аттрибуты'),
 ('включала', 'грецкие', 'орехи'),
 ('воздушные', 'шары', 'требуются'),
 ('возникла', 'картина', 'воспоминаний'),
 ('восточному', 'инжирным', 'вареньем'),
 ('встречаются', 'зазвездившиеся', 'админы'),
 ('вторую', 'смену', 'обидел'),
 ('выполнения', 'церемонии', 'чаяналивания'),
 ('выступали', 'egeiro', 'project'),
 ('высшее', 'наслаждение', 'пробуйте'),
 ('выходи

In [53]:
finder1.nbest(trigram_measures.likelihood_ratio, 30)

[('остались', 'очень', 'довольны'),
 ('в', 'общем', 'целом'),
 ('в', 'общем', 'остались'),
 ('дорого', 'в', 'общем'),
 ('в', 'общем', 'прекрасно'),
 ('в', 'общем', 'придраться'),
 ('в', 'общем', 'вечер'),
 ('бесплатная', 'в', 'общем'),
 ('в', 'общем', 'зайдём'),
 ('в', 'общем', 'оплатили'),
 ('в', 'общем', 'порекомендую'),
 ('ведущий', 'в', 'общем'),
 ('заметна', 'в', 'общем'),
 ('заявлен', 'в', 'общем'),
 ('комки', 'в', 'общем'),
 ('лезло', 'в', 'общем'),
 ('лучшем', 'в', 'общем'),
 ('одних', 'в', 'общем'),
 ('поднебесной', 'в', 'общем'),
 ('прочерк', 'в', 'общем'),
 ('развились', 'в', 'общем'),
 ('телефончик', 'в', 'общем'),
 ('в', 'общем', 'это'),
 ('в', 'общем', 'впечатления'),
 ('вкусный', 'в', 'общем'),
 ('кальян', 'в', 'общем'),
 ('голос', 'в', 'общем'),
 ('ожидании', 'в', 'общем'),
 ('фамилии', 'в', 'общем'),
 ('в', 'общем', 'очень')]

In [ ]:
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2), min_df=1, tokenizer=None, preprocessor=None,
                     stop_words=None, max_features=5000)
X_train_bow_ngrams = bigram_vectorizer.fit_transform(clean_texts_train).toarray()